In [1]:
import pandas as pd
import numpy as np

    Preprocessing

In [2]:
all_rooms = []
all_slots = []

In [3]:
df = pd.read_excel("TimeTable, FSC, Spring-2023.xlsx",sheet_name="Monday")
df = df.iloc[:39]       #Remove Labs from dataFrame

df = df.drop(columns=df.filter(like='Unnamed').columns)

In [4]:
monday = df["Monday"].to_list()

for i in monday:
    if str(i) == "nan" or str(i) == "Room":
        continue
    else:
        all_rooms.append(i)

In [5]:
row = df.iloc[[3]]
cols = df.columns.to_list()[1:]

for i in cols:
    all_slots.append(list(row[i])[0])

all_slots.pop()
all_slots.pop()

'05:20-06:40 '

In [6]:
def correctFormatting(name):

    if name[-1] == " ":
        name = name.rstrip(name[-1])

    name = name.replace("/"," ").replace("  "," ")

    if name.find("(") == -1:
        return name
    else:
        x = name.split(" ")
        final_name = ""
        for i in x:
            if i[0] == "(":
                final_name = final_name.rstrip(final_name[-1])
                return final_name
            else:
                final_name += i
                final_name += " "


checking_dict = {}
def checkDict(name):
    if name in checking_dict:
        return True
    else:
        return False

def correctCourseName(name):
    flag = checkDict(name)

    if flag:
        return checking_dict[name]


    x = name.split(' ')
    final_name = ""

    if len(x) == 1:
        return name
    
    for y in range(0,len(x)):

        if x[y].lower() == "and" or x[y].lower() == "of" or x[y].lower() == "to":
            continue

        if x[y].lower() == "database":
            final_name += "DB"
            continue

        if x[y].lower() == "mlops" or x[y].lower() == "devops":
            return name
            

        final_name += x[y][0]

    return final_name.upper()

    Computing Courses

In [7]:
computing_df = pd.read_csv("computing.csv")

computing_df = computing_df.iloc[1:]
computing_df.columns = computing_df.iloc[0]
computing_df = computing_df.iloc[1:]
computing_df = computing_df.iloc[:, : 6]

computing_df.drop(columns=["CHs","Code"],axis=1,inplace=True)

In [8]:
comp_trans_dfs = []
curs = np.nan
batch = np.nan

for i in range(0,computing_df.shape[0]):
    row = computing_df.iloc[i]

    if str(row["S.#"]) != "nan":    #Section Started
        batch = str(row["S.#"])
        for_name = correctFormatting(str(computing_df.iloc[i+1]["Course"]))
        curs = correctCourseName(for_name)

    else:
        if str(row["Course"]).find("MS Thesis") == -1:
            
            section = str(row["Section"])        
            inst = str(row["Course Instructor"])

            if str(row["Course"]) == "nan":
                comp_trans_dfs.append((curs,section,inst,batch))
            else:
                for_name = correctFormatting(str(row["Course"]))
                curs = correctCourseName(for_name)
                comp_trans_dfs.append((curs,section,inst,batch))

    SH Courses

In [9]:
sh_df = pd.read_csv("sh.csv")

sh_df = sh_df.iloc[1:]
sh_df.columns = sh_df.iloc[0]
sh_df = sh_df.iloc[1:]
sh_df = sh_df.iloc[:, : 6]

sh_df.drop(columns=["CHs","Code"],axis=1,inplace=True)
sh_df = sh_df.iloc[:123]

In [10]:
sh_trans_dfs = []
curs = np.nan
batch = np.nan

for i in range(0,sh_df.shape[0]):
    row = sh_df.iloc[i]

    if str(row["S.#"]) != "nan":    #Section Started
        batch = str(row["S.#"])
        for_name = correctFormatting(str(sh_df.iloc[i+1]["Course"]))
        curs = correctCourseName(for_name)

    else:

        section = str(row["Section"])        
        inst = str(row["Course Instructor"])

        if str(row["Course"]) == "nan":

            sh_trans_dfs.append((curs,section,inst,batch))
        else:
            for_name = correctFormatting(str(row["Course"]))
            curs = correctCourseName(for_name)
            sh_trans_dfs.append((curs,section,inst,batch))

    MG Courses

In [11]:
mg_df = pd.read_csv("mg.csv")

mg_df = mg_df.iloc[1:]
mg_df.columns = mg_df.iloc[0]
mg_df = mg_df.iloc[1:]
mg_df = mg_df.iloc[:, : 6]

mg_df.drop(columns=["CHs","Code"],axis=1,inplace=True)

In [12]:
mg_trans_dfs = []
curs = np.nan
batch = np.nan

for i in range(0,mg_df.shape[0]):
    row = mg_df.iloc[i]

    if str(row["S.#"]) != "nan":    #Section Started
        batch = str(row["S.#"])
        for_name = correctFormatting(str(mg_df.iloc[i+1]["Course"]))
        curs = correctCourseName(for_name)

    else:

        section = str(row["Section"])        
        inst = str(row["Course Instructor"])

        if str(row["Course"]) == "nan":

            mg_trans_dfs.append((curs,section,inst,batch))
        else:
            for_name = correctFormatting(str(row["Course"]))
            curs = correctCourseName(for_name)
            mg_trans_dfs.append((curs,section,inst,batch))

In [13]:
final_allele = []

for i in comp_trans_dfs:
    final_allele.append(i)

for i in sh_trans_dfs:
    final_allele.append(i)

for i in mg_trans_dfs:
    final_allele.append(i)

In [14]:
df = pd.DataFrame()

df["Allele"] = final_allele

df.to_csv("allele.csv")

In [15]:
df = pd.DataFrame()

df["Rooms"] = all_rooms

df.to_csv("rooms.csv")

df = pd.DataFrame()

df["Slots"] = all_slots

df.to_csv("slots.csv")